In [11]:
import pandas as pd
def make_ques_doc(experiment):
    human_data_file = pd.read_csv(experiment + "_counts.csv")
    
    with open("words_" + experiment + ".txt", 'w') as new_file:
        new_file.truncate(0)
        for i in range(len(human_data_file["prompt"])):
            new_file.write(human_data_file["type"][i] + " ")
            new_file.write(human_data_file["prompt_w1"][i] + " ")
            new_file.write(human_data_file["prompt_w2"][i] + " ")
            new_file.write(human_data_file["prompt_w3"][i] + " ")

            for r in human_data_file.columns[5:]:
                if human_data_file[r][i] > 0:
                    new_file.write(r + " ")
            new_file.write("\n")

        new_file.close()

make_ques_doc("experiment1a")
make_ques_doc("experiment1b")
make_ques_doc("experiment1c")

In [12]:
#read in csv file
#open text file with PG val from first column of csv file

# import os
# import shutil as sh

# def create_text_folder(genre):
#     corpus = pd.read_csv(genre + "_texts.csv")
#     texts = corpus["id"]
    
#     #path will be changed
#     path = genre + "_docs"
#     # Check whether the specified path exists or not
#     isExist = os.path.exists(path)
#     if not isExist:
#         os.makedirs(path)
#     for i in range(len(texts)):
#         sh.copy("/Users/erikajones/Documents/CS 8803/LSA Project/text_database/gutenberg/data/text/" + texts[i] + "_text.txt", path)

# def create_token_folder(genre):
#     corpus = pd.read_csv(genre + "_texts.csv")
#     texts = corpus["id"]
    
#     #path will be changed
#     path = genre + "_tokens"
#     # Check whether the specified path exists or not
#     isExist = os.path.exists(path)
#     if not isExist:
#         os.makedirs(path)
#     for i in range(len(texts)):
#         sh.copy("/Users/erikajones/Documents/CS 8803/LSA Project/text_database/gutenberg/data/tokens/" + texts[i] + "_tokens.txt", path)
        
# create_text_folder("nonfiction")
# create_text_folder("fiction")
# create_text_folder("poetry")

# create_token_folder("nonfiction")
# create_token_folder("fiction")
# create_token_folder("poetry")
        

In [13]:
import glob
def find_analogy_questions(experiment, genre):
        
    corpus = pd.read_csv(genre + "_texts.csv")
    texts = corpus["id"]
    
    with open(experiment + "_" + genre + "_analogy_questions.txt", "w") as analogies:
        analogies.truncate(0)
        f = open("words_" + experiment + ".txt", 'r')
        questions = f.readlines()
        f.close()
        questions_dict = {}
        words_in_doc = True
        
        for row in questions:
            row = row.split()
            line = row[1:9]
            words_in_docs = True
            count_sum = 0
            word_info = []
            
            if row[0] not in questions_dict.keys():
                questions_dict[row[0]] = []
            
            for word in line:
                
                word_count = 0
                for i in range(len(texts)):
                    with open(genre + "_tokens/" + texts[i] + "_tokens.txt", "r") as file:
                        tokens = file.read()
                        if word in tokens:
                            word_count += 1
                        file.close()
                if word_count == 0:
                    words_in_docs = False
                
                word_info.append(word) 
                count_sum += word_count
                
            while len(word_info) < 8:
                word_info += [""]
            if words_in_docs:
                count_sum = [count_sum]
                questions_dict[row[0]].append(count_sum + word_info)
                questions_dict[row[0]].sort()
            
        for i in questions_dict.items():
            analogies.write(str(i) + "\n")
            
        analogies.close()
        return questions_dict



In [15]:
experiments = ["experiment1a", "experiment1b", "experiment1c"]
genres = ["nonfiction", "fiction", "poetry"]

analogy_list = []
for e in experiments:
    analogy_list_byexp = []
    for g in genres:
        analogy_list_byexp.append(find_analogy_questions(e,g))
    analogy_list.append(analogy_list_byexp)
        

In [24]:
index = 0
final_question_dict = {}
for exps in analogy_list:
    
    final_question_dict[experiments[index]] = []
    question_dict = {}
    
    for k in exps[0].keys():
        
        question_dict[k] = []
            
        for q_nf in exps[0][k]:
            in_fiction = False
            in_poetry = False

            for q_f in exps[1][k]:
                if q_nf[1:4] == q_f[1:4]:
                    in_fiction = True

            for q_p in exps[2][k]:
                if q_nf[1:4] == q_p[1:4]:
                    in_poetry = True

            if in_fiction and in_poetry:
                question_dict[k].append(q_nf[1:9])
    final_question_dict[experiments[index]] = question_dict
    index += 1
                    
                    

In [27]:
import os
final_dict = {}

for exp in experiments:
    type_dict = {}
    for types in final_question_dict[exp].keys():
        type_dict["prompt1"] = [col[0] for col in final_question_dict[exp][types]]
        type_dict["prompt2"] = [col[1] for col in final_question_dict[exp][types]]
        type_dict["prompt3"] = [col[2] for col in final_question_dict[exp][types]]
        type_dict["response1"] = [col[3] for col in final_question_dict[exp][types]]
        type_dict["response2"] = [col[4] for col in final_question_dict[exp][types]]
        type_dict["response3"] = [col[5] for col in final_question_dict[exp][types]]
        type_dict["response4"] = [col[6] for col in final_question_dict[exp][types]]
        type_dict["response5"] = [col[7] for col in final_question_dict[exp][types]]
        
        df = pd.DataFrame(type_dict)
        
        path = "analogy_question_selections"
        # Check whether the specified path exists or not
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)
        df.to_csv(path + "/" + exp + "_" + types + "_questions.csv")
    

In [9]:
with open("fiction_tokens/" + "PG10002_tokens.txt", "r") as file:
    tokens = file.read()
#     print(tokens)
    word_count = 0
    if "the" in tokens:
        word_count += 1
        print(word_count)

1
